In [3]:
import os
import yaml

year1 = 1990
year2 = 1997

# Directory containing the YAML files
yaml_dir = './ecmean'

# Dictionary to hold the contents of the YAML files
stat = {}

# Loop over all files in the directory
for filename in os.listdir(yaml_dir):
    if filename.endswith(f"{year1}_{year2}.yml"):
        # Extract the varying part of the filename as the key
        key = filename.split("_")[2]  # Assuming the key is always in the same position
        # Construct the full path to the file
        filepath = os.path.join(yaml_dir, filename)
        # Open and load the YAML file
        with open(filepath, 'r') as file:
            stat[key] = yaml.safe_load(file)



In [4]:
targets = ['net_toa', 'rsnt', 'rlnt', 'swcf', 'lwcf', 'rsns', 'rlns', 'hfss', 'hfls', 'net_sfc', 'toamsfc']

In [5]:
#               RPRCON: 0.14E-02    # coefficients for determining conversion from cloud water to rain
#               ENTRORG: 0.175E-02  # entrainment rate (positive buoyant convection)
#               DETRPEN: 0.75E-04   # detrainment rate for penetrative convection
#               ENTRDD: 0.3E-03     # entrainment rate for cumulus downdrafts
#               RMFDEPS: 0.3        # fractional massflux for downdrafts at lfs
#           namcldp:
#               RVICE: 0.13         # fixed ice fallspeed
#               RLCRITSNOW: 0.3E-04 # critical autoconversion threshold
#               RSNOWLIN2: 0.3E-01  # Constant governing of the temperature dependence of the autoconversion
#                                   # of ice crystals to snow in large-scale precipitation
#               RCLDIFF: 0.3E-05    # diffusion-coefficient for evaporation by turbulent mixing
#               RCLDIFF_CONVI: 7.0  # enhancement factor of rcldiff for convection

pars = ['RPRCON', 'ENTRORG', 'DETRPEN', 'ENTRDD', 'RMFDEPS', 'RVICE', 'RLCRITSNOW', 'RSNOWLIN2', 'RCLDIFF', 'RCLDIFF_CONVI']
vals = [0.14E-02, 0.175E-02, 0.75E-04, 0.3E-03, 0.3, 0.13, 0.3E-04, 0.3E-01, 0.3E-05, 7.0]
inc=0.2

In [9]:
import numpy as np

sensitivity = {}

for p in range(len(pars)):
    sensitivity[pars[p]] = {}
    st0 = stat['s000']
    st1 = stat[f's0{p}1']
    st2 = stat[f's0{p}2']

    for key1 in targets:
        sensitivity[pars[p]][key1] = {}

        for key2 in st0[key1]:
            sensitivity[pars[p]][key1][key2] = {}

            for key3 in st0[key1][key2]:
                am = st1[key1][key2][key3]
                a0 = st0[key1][key2][key3]
                ap = st2[key1][key2][key3]
                vm = vals[p] * (1 - inc)
                v0 = vals[p]
                vp = vals[p] * (1 + inc)
                x = np.array([vm, v0, vp])
                y = np.array([am, a0, ap])
                coefficients = np.polyfit(x, y, 1)
                sensitivity[pars[p]][key1][key2][key3] = [float(coefficients[0]),
                                                          float(coefficients[0])*v0]
            


In [10]:
import yaml

output_file = f'sensitivity_{year1}-{year2}.yaml'

# Save the sensitivity dictionary into the YAML file
with open(output_file, 'w') as file:
    yaml.dump(sensitivity, file)

In [10]:
import numpy as np

paramdic = {}
outdir = './exps'
prefix = 's0'

for p in range(len(pars)):
    paramdic[pars[p]] = vals[p]

fn = f'params_{prefix}00.yaml'
filepath = os.path.join(outdir, fn)
with open(filepath, 'w') as file:
    yaml.dump(paramdic, file)

for p in range(len(pars)):

    paramdic2 = paramdic.copy()
    
    fn = f'params_{prefix}{p}1.yaml'
    v = vals[p] * (1 - inc)
    paramdic2[pars[p]] = v

    filepath = os.path.join(outdir, fn)
    with open(filepath, 'w') as file:
        yaml.dump(paramdic2, file)
    
    paramdic2 = paramdic.copy()
    
    fn = f'params_{prefix}{p}2.yaml'
    v = vals[p] * (1 + inc)
    paramdic2[pars[p]] = v

    filepath = os.path.join(outdir, fn)
    with open(filepath, 'w') as file:
        yaml.dump(paramdic2, file)


     

            


In [11]:
['RPRCON', 'ENTRORG', 'DETRPEN', 'ENTRDD', 'RMFDEPS', 'RVICE', 'RLCRITSNOW', 'RSNOWLIN2', 'RCLDIFF', 'RCLDIFF_CONVI']

['RPRCON',
 'ENTRORG',
 'DETRPEN',
 'ENTRDD',
 'RMFDEPS',
 'RVICE',
 'RLCRITSNOW',
 'RSNOWLIN2',
 'RCLDIFF',
 'RCLDIFF_CONVI']